In [ ]:
!pip install learn2learn

     |████████████████████████████████| 604 kB 29.7 MB/s 
     |████████████████████████████████| 2.9 MB 35.1 MB/s 
     |████████████████████████████████| 135 kB 55.6 MB/s 
     |████████████████████████████████| 96 kB 7.0 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 1.4 MB 24.9 MB/s 
     |████████████████████████████████| 3.5 MB 47.8 MB/s 
  Created wheel for learn2learn: filename=learn2learn-0.1.6-cp37-cp37m-linux_x86_64.whl size=945738 sha256=38305983df2f9e815fdc651b24a82481b011d2e377bceba58aaedbcc87f673dc
  Stored in directory: /root/.cache/pip/wheels/4f/d0/99/d65b01d3a6bda35325b9b2e964c7c17169951c915484029b74
  Created wheel for qpth: filename=qpth-0.0.15-py3-none-any.whl size=15379 sha256=432231ad72967aa2f8f8f63d3a3af4aa1d68608ecb87e1e95d8b4904d581391f
  Stored in directory: /root/.cache/pip/wheels/f6/bb/0f/3af358159c8cfc56654d85ba5069b53ab351dee72f5a57c2ff
  Created wheel for gsutil: filename=gsutil-5.4-py3-none-any.whl 

In [ ]:
import torch
import torchvision
import torch.nn as nn
import learn2learn as l2l
from torch.utils.data import Dataset,DataLoader
import random
import torchvision.transforms as transforms

In [ ]:
class make_dataset(Dataset):
  def __init__(self,data,selection):
    super().__init__()
    self.data = data
    self.selection = selection
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self,i):
    a = self.data[i]
    try:
      return a[0][0],self.selection.index(a[0][1])
    except:
      return a[0],self.selection.index(a[1])

class data(Dataset):
  def __init__(self):
    super().__init__()
    d = torchvision.datasets.MNIST('/',download = True,transform=transforms.ToTensor())
    self.l = {}
    for i in range(len(d)):
      x = d[i]
      try:
        self.l[x[1]].append([x])
      except:
        self.l[x[1]] = [x]
  
  def __len__(self):
    return 1000
  def __getitem__(self,i):
    if(i>=len(self)):
      raise IndexError('list index out of range')
    meta_train = []
    meta_test = []
    selection = random.sample(list(range(10)),5)
    selection.sort()
    for i in selection:
      for x in random.sample(self.l[i],5):
        meta_train.append(x)
      for x in random.sample(self.l[i],1):
        meta_test.append(x)
    meta_train = make_dataset(meta_train,selection)
    meta_test = make_dataset(meta_test,selection)
    return meta_train,meta_test

dataset = data()

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(784,512)
    self.l2 = nn.Linear(512,256)
    self.l3 = nn.Linear(256,128)
    self.l4 = nn.Linear(128,64)
    self.l5 = nn.Linear(64,5)
    self.relu = nn.LeakyReLU()
  def forward(self,x):
    x = self.l1(x)
    x = self.relu(x)
    x = self.l2(x)
    x = self.relu(x)
    x = self.l3(x)
    x = self.relu(x)
    x = self.l4(x)
    x = self.relu(x)
    x = self.l5(x)
    return x

model = Network()

In [ ]:
meta_train_loss = nn.CrossEntropyLoss()
meta_test_loss = nn.CrossEntropyLoss()

In [ ]:
maml = l2l.algorithms.MAML(model,0.001)
opt = torch.optim.Adam(maml.parameters(),lr = 0.001)
epochs = 100
for i in range(epochs):
  acc = 0
  post_step_acc = 0
  for j,(x,y) in enumerate(dataset):
    metatrain = DataLoader(x,len(x),True)
    metatest = DataLoader(y,len(y),True)
    for k,(xtr,ytr) in enumerate(metatrain):
      opt.zero_grad()
      task_model = maml.clone()
      xtr = xtr.flatten(1)
      y_pred = task_model(xtr) 
      adaptation_loss = meta_train_loss(y_pred,ytr)
      with torch.no_grad():
        for k,(xtr,ytr) in enumerate(metatest):
          xtr = xtr.flatten(1)
          y_pred_acc = task_model(xtr)
          acc+= ((y_pred_acc==ytr).to(dtype = torch.float32)).mean()    
      task_model.adapt(adaptation_loss)
    for k,(xtr,ytr) in enumerate(metatest):
      xtr = xtr.flatten(1)
      y_pred = task_model(xtr)
      evaluation_loss = meta_test_loss(y_pred,ytr)
      with torch.no_grad():
          post_step_acc+= ((y_pred==ytr).to(dtype = torch.float32)).mean()
      evaluation_loss.backward()
      opt.step()
    print(f'epoch: {i} batch_no: {j} adaption_loss: {adaptation_loss}  eval_loss: {evaluation_loss}  drop_in_step: {adaptation_loss-evaluation_loss}')
  print(f'epoch: {i} adaption_loss: {adaptation_loss}  eval_loss: {evaluation_loss}  drop_in_step: {adaptation_loss-evaluation_loss} accuracy: {acc} post step accuracy: {post_step_acc}')

epoch: 0 batch_no: 0 adaption_loss: 4.423384666442871  eval_loss: 0.011505749076604843  drop_in_step: 4.411879062652588
epoch: 0 batch_no: 1 adaption_loss: 4.581888198852539  eval_loss: 0.00022848467051517218  drop_in_step: 4.58165979385376
epoch: 0 batch_no: 2 adaption_loss: 8.995071411132812  eval_loss: 3.311399632366374e-05  drop_in_step: 8.995038032531738
epoch: 0 batch_no: 3 adaption_loss: 4.936190128326416  eval_loss: 0.006358731538057327  drop_in_step: 4.929831504821777
epoch: 0 batch_no: 4 adaption_loss: 5.324308395385742  eval_loss: 0.00011127337347716093  drop_in_step: 5.324197292327881
epoch: 0 batch_no: 5 adaption_loss: 5.860894203186035  eval_loss: 0.0036052637733519077  drop_in_step: 5.857288837432861
epoch: 0 batch_no: 6 adaption_loss: 5.376147270202637  eval_loss: 0.6999979019165039  drop_in_step: 4.676149368286133
epoch: 0 batch_no: 7 adaption_loss: 8.52431583404541  eval_loss: 0.44014349579811096  drop_in_step: 8.084172248840332
epoch: 0 batch_no: 8 adaption_loss: 9.5

KeyboardInterrupt: ignored